#intro here

In [2]:
import pandas as pd

df = pd.read_csv("dataset/orders.csv")

# Quick overview
print(df.info())
print(df.describe())

# Frequency counts
print(df['eval_set'].value_counts())
print(df['order_dow'].value_counts(normalize=True))

# Missing values
print(df.isna().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB
None
           order_id       user_id  order_number     order_dow  \
count  3.421083e+06  3.421083e+06  3.421083e+06  3.421083e+06   
mean   1.710542e+06  1.029782e+05  1.715486e+01  2.776219e+00   
std    9.875817e+05  5.953372e+04  1.773316e+01  2.046829e+00   
min    1.000000e+00  1.000000e+00  1.000000e+00  0.000000e+00   
25%    8.552715e+05  5.139400e+04  5.000000e+00  1.000000e+00   
50%    1.710542e+06  1.026890e+05  1.100000e+01  3.000000e+00   
75%    2.565812e+06  1.543850e+

In [ ]:


sample_df = df.sample(50000)
profile = ProfileReport(sample_df, explorative=True)
profile.to_file("sample_report.html")


In [2]:
from ydata_profiling import ProfileReport
import pandas as pd

orders = pd.read_csv('dataset/orders.csv')
order_products_prior = pd.read_csv('dataset/order_products__prior.csv')
products = pd.read_csv('dataset/products.csv')
aisles = pd.read_csv('dataset/aisles.csv')
departments = pd.read_csv('dataset/departments.csv')

products_full = (products
    .merge(aisles, on='aisle_id', how='left')
    .merge(departments, on='department_id', how='left')
)

prior_products = order_products_prior.merge(products_full, on='product_id', how='left')

full_orders_products = prior_products.merge(
    orders,
    on='order_id',
    how='left'
)

# Don't need these since we have the descriptions
columns_to_remove = ["order_id", "product_id", "aisle_id", "department_id", "eval_set"]

full_orders_products_df = full_orders_products.drop(columns=columns_to_remove)

# Build a sample of users to keep basket integrity and product diversity
sample_frac = 0.05
sampled_users = orders['user_id'].drop_duplicates().sample(frac=sample_frac, random_state=42)
df_sample = full_orders_products_df[full_orders_products_df['user_id'].isin(sampled_users)]

profile = ProfileReport(df_sample, explorative=True)
profile.to_file("eda_report_orders.html")

c:\Users\jenle\AppData\Local\Programs\Python\Python312\Lib\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:18<00:00,  1.87s/it]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]